# Задание 10

Бинаризовать нейронную сеть (полносвязный слой), доступен `torch==1.5.1`

Вывести градиенты для введенных весов и входов

In [ ]:
import sys
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

## Inputs/Outputs

Expected input:
```
4
3
-0.1924 -0.7276 0.6216
-0.2294 -0.1115 -2.1671
-1.3199 0.5962 -0.3633
-0.4749 1.4275 -1.4095
5
3
1.4425 0.2085 -1.0470
0.3339 0.0046 -1.8452
-1.2203 -0.1503 -0.5810
0.3694 -0.8143 0.8213
2.4192 -1.2167 -1.2936
```

Expected output:
```
4.8456 -0.5448 -2.2704
4.6236 -1.7770 0.0000
0.0000 -0.8076 -3.8202
3.1506 0.0000 0.0000
-4.0 0.0 -2.0
-4.0 0.0 -2.0
-4.0 0.0 -2.0
-4.0 0.0 -2.0
-4.0 0.0 -2.0
```

In [ ]:
tensor_shape = (int(input()), int(input()))
X = [list(map(float, input().split())) for x_str in range(tensor_shape[0])]
feats_out, feats_in = (int(input()), int(input()))
W = [list(map(float, input().split())) for feats_str in range(feats_out)]
y = np.zeros((tensor_shape[0], feats_out))

X = torch.tensor(X)
W = torch.tensor(W)
y = torch.tensor(y)


def print_grad(tensor):
    np.savetxt(sys.stdout, tensor.grad.numpy(), fmt='%.04f')

## Бинаризаторы

$$
f(x) = 
  \begin{cases}
    −1         & \quad \text{при } x \lt 0 \\
    1          & \quad \text{при } x \ge 0
  \end{cases}
$$

Из-за этого неудобно использовать функцию `torch.sign`, так как для `0` она возвращает `0`, а не `1`

Для входов бинаризация производится с использованием `STESignApproximator`. Производная в этом случае считается за единицу

Для входов бинаризация производится с использованием `ParabolaSignApproximator`. Бинаризация апроксимируется параболой следующим образом:

$$
f(x) = 
  \begin{cases}
    −1         & \quad \text{при } x \lt −1 \\
    x^2 + 2x   & \quad \text{при } −1 \le x \lt 0 \\
    -x^2 + 2x  & \quad \text{при } 0 \le x \lt 1 \\
    1          & \quad \text{при } x \ge 1
  \end{cases}
$$

Производная выражается следующим образом:

$$
f'(x) = 
  \begin{cases}
    0         & \quad \text{при } x \lt −1 \\
    2x + 2    & \quad \text{при } −1 \le x \lt 0 \\
    -2x + 2   & \quad \text{при } 0 \le x \lt 1 \\
    0         & \quad \text{при } x \ge 1
  \end{cases}
$$

In [ ]:
class ParabolaSignApproximator(torch.autograd.Function):
    @staticmethod
    def forward(ctx, x):
        ctx.save_for_backward(x)
        values = x.detach().numpy()
        return x.new(np.where(values < 0, -1, 1))

    @staticmethod
    def backward(ctx, grad_output):

        x = ctx.saved_tensors[0].detach().numpy()
        grads = grad_output.detach().numpy()

        grad_input = np.zeros_like(grad_output)
        mask = (-1 <= x) & (x < 0)
        grad_input[mask] = grads[mask] * (2 * x[mask] + 2)

        mask = (0 <= x) & (x < 1)
        grad_input[mask] = grads[mask] * (-2 * x[mask] + 2)
        return grad_output.new(grad_input)


class STESignApproximator(torch.autograd.Function):
    @staticmethod
    def forward(ctx, x):
        values = x.detach().numpy()
        return x.new(np.where(values < 0, -1, 1))

    @staticmethod
    def backward(ctx, grad_output):
        return grad_output

Для ответа нужно вывести градиенты после одного прохода данных и весов через полносвязный слой без смещения

В качестве функции потерь используется выход полносвязного слоя

In [ ]:
weights_binary = STESignApproximator()
inputs_binary = ParabolaSignApproximator()

inputs = torch.clone(X)
inputs.requires_grad = True
weights = nn.Parameter(data=W, requires_grad=True)

weights_bin = weights_binary.apply(weights)
inputs_bin = inputs_binary.apply(inputs)

outputs = F.linear(inputs_bin, weights_bin, None)
torch.sum(outputs).backward()

print_grad(inputs)
print_grad(weights)